In [1]:
using Revise
includet("vtol.jl")
includet("adaptation.jl")
includet("high_level_control.jl")
includet("control_allocation.jl")
includet("simulation.jl")
import .ControlAllocation: n_x, n_u, n_λ, n_W
import .HighLevelController: ref_pose, ref_velocity, RefTrajParams
import .Simulation: SimulationData, SimulationParams, dudt!, adaptive_control_allocator!
import .VTOL: VTOLParams
using LinearAlgebra, Plots, DifferentialEquations, ForwardDiff, StaticArrays, DiffEqCallbacks

In [3]:
x0 = @MVector zeros(n_x);

sim = SimulationParams()

r0 = ref_pose(sim.t0, RefTrajParams())
rdot0 = ref_velocity(sim.t0, RefTrajParams())
x0[1] = 0.0 + r0[1]  # Initial position x
x0[4] = 0.0 + rdot0[1]  # Initial velocity x
x0[2] = 1.0 + r0[2]  # Initial position y
x0[5] = 0.0 + rdot0[2]  # Initial velocity y
x0[3] = r0[3]+deg2rad(4.0)  # Initial pitch angle

xhat0 = x0

λ = @MVector zeros(n_λ);
u = @MVector[0.01, 0.01, 0.4, -0.3, x0[3], 0.0, 0.0];
vtol = VTOLParams()
W_true = @SVector[vtol.CDδ, vtol.CDα, vtol.CDt, vtol.CLδ, vtol.CLα, vtol.Cmδ, vtol.Cmα, 1.0, 0.0]
W = MVector{n_W}(W_true*1.5)

cb = PeriodicCallback(adaptive_control_allocator!, sim.dt;initial_affect=true)

u_hist = [u]
λ_hist = [λ]
W_hist = [W]
xhat_hist = [xhat0]
t_hist = [sim.t0]

data = SimulationData(u=u, λ=λ, W=W, xhat=xhat0)

prob = ODEProblem(dudt!, x0, (sim.t0, sim.t_final), data)

@time sol = solve(prob, Tsit5(), callback = cb)

retcode: Success
Interpolation: specialized 4th order "free" interpolation
t: 6081-element Vector{Float64}:
  0.0
  0.0
  7.911899870101969e-7
  6.63946149609928e-6
  2.0067741536143656e-5
  3.9476397856247065e-5
  6.436212906548743e-5
  9.658193581853715e-5
  0.00013649786629785533
  0.0001856725000046004
  ⋮
 29.96
 29.96
 29.97
 29.97
 29.98
 29.98
 29.990000000000002
 29.990000000000002
 30.0
u: 6081-element Vector{MVector{6, Float64}}:
 [-260.0111268042049, 101.00556340210242, 0.06981317007977318, 19.985219285847666, 0.009244824443644162, 0.0]
 [-260.0111268042049, 101.00556340210242, 0.06981317007977318, 19.985219285847666, 0.009244824443644162, 0.0]
 [-260.01111099294667, 101.00556341824921, 0.06981317008907804, 19.98307379170736, 0.03152937830221407, 2.3618751495936846e-5]
 [-260.0109941742849, 101.00556406842884, 0.06981317077435793, 19.966549759068442, 0.18861912733207498, 0.00021580961378575323]
 [-260.01072633042236, 101.00556878035007, 0.06981317719411102, 19.9253491712833

In [ ]:
t_vec = data.t_hist
τ = zeros(n_λ,length(t_vec))

L_vec = zeros(length(t_vec))

J_vec = zeros(length(t_vec))

k_vec = zeros(n_λ,length(t_vec))

FOO_vec = zeros(n_u+n_λ, length(t_vec))

M_aero_vec = zeros(length(t_vec))
α_vec = zeros(length(t_vec))
e_s_vec = zeros(6, length(t_vec))
norm_e_s_vec = zeros(length(t_vec))

uλWdot_vec = zeros(n_u+n_λ+n_W, length(t_vec)-1)

Lift_vec = zeros(length(t_vec))

lyapunov_vec = zeros(length(t_vec))
Vdot_vec = zeros(length(t_vec))
# Γ_ϵ = Diagonal([10.0, 10.0, 10.0, 10.0, 100.0, 100.0])

sliding_vec = zeros(n_λ, length(t_vec))
du = zeros(length(sol(0)))

u_hist = zeros(n_u, length(t_vec))
λ_hist = zeros(n_λ, length(t_vec))
W_hist = zeros(n_W, length(t_vec))


norm_W_vec = zeros(length(t_vec))

vel_mag_vec = zeros(length(t_vec))

d2Ldxdudλ_vel_vec = zeros(length(t_vec))

for (j, t) in enumerate(t_vec)
    x = sol(t)
    u = data.u_hist[j]
    λ = data.λ_hist[j]
    W = data.W_hist[j]
    xhat = data.xhat_hist[j]
    u_hist[:,j] = u[1:end]
    λ_hist[:,j] = λ[1:end]
    W_hist[:,j] = W[1:end]
    norm_W_vec[j] = norm(W)
    vel_mag_vec[j] = sqrt(x[4]^2 + x[5]^2)
    # W_reshape = reshape(W, 5, n_λ)
    # NN = W_reshape' * [Φ_Nn_W(x,u, u[5],model);1.0] ./scale
    # NN_hist[:,j] = NN
    # V = sqrt(x[4]^2 + x[5]^2)
    # NN_truth_hist[:,j] = Φ_sim(V, α(x,x[3]), x[6], u[1:4], α(x,u[5]))
    ϕ = Φ_full(t,x, u, W_true)
    z = data.z_hist[j]
    e_s_vec[:,j] = x - xhat
    norm_e_s_vec[j] = norm(e_s_vec[:,j])
    τ[:,j] = ϕ
    # L_vec[j] = L_flat(vcat(sol(t), t), n_x, n_u, n_λ, n_W)
    J_vec[j] = J(t, x, u)
    k_vec[:,j] = k(t,x, u,z,xhat)-[u[6];u[7];0]
    FOO_vec[:,j] = [dL_du(t, x, u, λ, W, z, xhat); constraint(t, x, u, λ, W, z, xhat)]
    α_vec[j] = α(x, x[3])
    if j < length(t_vec) 
        uλWdot_vec[:,j] = uλW_dot(t, x, u, λ, W, z, xhat,data)
    end

    dLduλ = [dL_du(t,x,u,λ,W,z,xhat); constraint(t,x,u,λ,W,z,xhat)]
    d2Ldu = dL_du_jacobian(t, x, u, λ, W, z, xhat)
    d2Ldλ = dL_dλ_jacobian(t, x, u, λ, W, z, xhat)

    d2Ldxdudλ = [d2Ldu[:,SA[1:n_x...]]; d2Ldλ[:,SA[1:n_x...]]]

    d2Ldxdudλ_vel_vec[j] = norm(d2Ldxdudλ[:,SA[4:6...]])
    # M_aero_vec[j] = M_aero_true(x, u, α_vec[j])
    # dudt!(du,sol(t),result,t)
    # actuator_rate_vec[:,j] = du[n_x+1:n_x+n_u]
    # Lift_vec[j] = F_aero(x, u, α_vec[j])[2]
    # s = sliding_surface(t, x, u[5],z)
    # sliding_vec[:,j] = s
    # lyapunov_vec[j] = 0.5*FOO_vec[:,j]'*FOO_vec[:,j] + 0.5*(W - vec(W_NN_0))'/Γ_W_inv*(W - vec(W_NN_0)) + 0.5*s'*M*s
    # Vdot_vec[j] = -s'*K*s - s'*FOO_vec[n_u+1:end,j] - FOO_vec[:,j]'*10*FOO_vec[:,j]
end



UndefVarError: UndefVarError: `n_θ` not defined in `Main`
Suggestion: check for spelling errors or missing imports.